# Jane Street Market Prediction
[Kaggle Jane Street Market Prediction](https://www.kaggle.com/c/jane-street-market-prediction/overview)


In [1]:
import pandas as pd
import numpy as np
import datatable as dt
X = dt.fread('jane-street-market-prediction/train.csv').to_pandas()

In [2]:
Xt = X[X.date >= 460]
yt = np.where((Xt.resp > 0), 1, 0)
X = X[(X.date > 85) & (X.date < 460)]
y = np.where((X.resp > 0), 1, 0)
drop_cols = ['date', 'resp_1', 'resp_2', 'resp_3', 'resp_4', 'resp', 'ts_id']
X.drop(drop_cols, axis=1, inplace=True)
Xt.drop(drop_cols, axis=1, inplace=True)

In [3]:
import lightgbm as lgb
lgb_train = lgb.Dataset(X, y)
lgb_eval = lgb.Dataset(Xt, yt, reference=lgb_train)

In [4]:
params = {
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': {'binary_logloss', 'auc'},
}

In [5]:
gbm = lgb.train(params,
                lgb_train,
                valid_sets=lgb_eval)

[LightGBM] [Info] Number of positive: 829802, number of negative: 819562
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 2.987766 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 33153
[LightGBM] [Info] Number of data points in the train set: 1649364, number of used features: 131
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.503104 -> initscore=0.012417
[LightGBM] [Info] Start training from score 0.012417
[1]	valid_0's binary_logloss: 0.693037	valid_0's auc: 0.517087
[2]	valid_0's binary_logloss: 0.692858	valid_0's auc: 0.519372
[3]	valid_0's binary_logloss: 0.692748	valid_0's auc: 0.520937
[4]	valid_0's binary_logloss: 0.692654	valid_0's auc: 0.522388
[5]	valid_0's binary_logloss: 0.692535	valid_0's auc: 0.522543
[6]	valid_0's binary_logloss: 0.692403	valid_0's auc: 0.52356
[7]	valid_0's binary_logloss: 0.692269	valid_0's auc: 0.525132
[8]	valid_0's binary_logloss: 0.692162	valid_0's auc: 0.525

In [6]:
y_pred = gbm.predict(Xt, num_iteration=gbm.best_iteration)

In [7]:
y_pred

array([0.49830597, 0.52623212, 0.49772581, ..., 0.49985955, 0.5743741 ,
       0.51928774])

In [8]:
from sklearn.metrics import mean_squared_error
print('The rmse of prediction is:', mean_squared_error(yt, y_pred) ** 0.5)

The rmse of prediction is: 0.499075174426875


In [13]:
y[:10]

array([0, 0, 0, 1, 0, 0, 0, 1, 0, 0])

In [14]:
y_pred[:10]

array([0.49830597, 0.52623212, 0.49772581, 0.50207263, 0.49008112,
       0.49510764, 0.49496341, 0.4992098 , 0.49443664, 0.50299461])